In [ ]:
# crear dstc2_dialogues_input_gpt2

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
import json
import os

from sklearn.preprocessing import LabelEncoder

from huggingface_hub import notebook_login
from datasets import Dataset, load_dataset

In [ ]:
notebook_login()

## 1) Load data

### 1.1) Text

In [ ]:
dataset = load_dataset('danielroncel/dstc2_dialogues_processed')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/22266 [00:00<?, ? examples/s]

In [ ]:
df = dataset['train'].to_pandas()
df.head()

,session_id,turn_index,audio_file,transcript,chat_history_last_9,chat_history_last_9_tokenized,speaker_text_last_9_tokenized,attention_mask,label_semantics,label
0,voip-00d76b791d-20130327_010416,2,pt344x_0000993_0001219.wav,<user>expensive restaurant in the south part o...,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|area_inform|pricerange,request|food
1,voip-00d76b791d-20130327_010416,4,pt344x_0001649_0001680.wav,<user>any<API_call><DB_result>,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|this,inform|area_inform|food_inform|pricerange_offe...
2,voip-00d76b791d-20130327_010416,6,pt344x_0002674_0002736.wav,<user>address<API_call><DB_result>,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",request|addr,inform|addr_offer|name
3,voip-00d76b791d-20130327_010416,8,pt344x_0003533_0003611.wav,<user>type of food<API_call><DB_result>,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",request|food,inform|area_inform|pricerange_offer|name
4,voip-00d76b791d-20130327_011116,2,pt344x_0001082_0001294.wav,<user>lebanese restaurant in the west part of ...,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|area_inform|food,request|food


## 1.2) Audios

In [ ]:
dataset = load_dataset('danielroncel/dstc2_audios')
dataset

Generating train split:   0%|          | 0/25516 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'session_ids'],
        num_rows: 25516
    })
})

In [ ]:
df_audios = dataset['train'].to_pandas()
df_audios.head()

,audio,session_ids
0,{'bytes': b'RIFF\xa4\x1a\x01\x00WAVEfmt \x10\x...,voip-00d76b791d-20130327_010416
1,{'bytes': b'RIFF\xe4&\x00\x00WAVEfmt \x10\x00\...,voip-00d76b791d-20130327_010416
2,{'bytes': b'RIFF\xa4M\x00\x00WAVEfmt \x10\x00\...,voip-00d76b791d-20130327_010416
3,{'bytes': b'RIFF\xa4a\x00\x00WAVEfmt \x10\x00\...,voip-00d76b791d-20130327_010416
4,{'bytes': b'RIFF\xa4a\x00\x00WAVEfmt \x10\x00\...,voip-00d76b791d-20130327_010416


In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    paths = [x["path"] for x in examples["audio"]]
    sampling_rate = [x["sampling_rate"] for x in examples["audio"]]

    return {'path': paths, 'array': audio_arrays, 'sampling_rate': sampling_rate}

dataset = dataset.map(preprocess_function, batched=True)
dataset

Map:   0%|          | 0/25516 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'session_ids', 'path', 'array', 'sampling_rate'],
        num_rows: 25516
    })
})

In [ ]:
dataset = dataset.remove_columns(["audio"])
dataset

DatasetDict({
    train: Dataset({
        features: ['session_ids', 'path', 'array', 'sampling_rate'],
        num_rows: 25516
    })
})

In [ ]:
df_audios = dataset['train'].to_pandas()
df_audios.head()

,session_ids,path,array,sampling_rate
0,voip-00d76b791d-20130327_010416,pt344x_0000993_0001219.wav,"[0.000335693359375, 0.000335693359375, 0.00012...",16000
1,voip-00d76b791d-20130327_010416,pt344x_0001649_0001680.wav,"[0.0001220703125, 0.000152587890625, 0.0003356...",16000
2,voip-00d76b791d-20130327_010416,pt344x_0002674_0002736.wav,"[0.000152587890625, 0.000152587890625, 0.00027...",16000
3,voip-00d76b791d-20130327_010416,pt344x_0003533_0003611.wav,"[0.000213623046875, 0.00054931640625, 0.000671...",16000
4,voip-00d76b791d-20130327_010416,pt344x_0004453_0004531.wav,"[0.000213623046875, 0.000335693359375, 0.00027...",16000


In [ ]:
df_audios = df_audios.rename({'session_ids': 'session_id',
                              'path': 'audio_file'}, axis=1)
df_audios = df_audios.drop('sampling_rate', axis=1)

In [ ]:
df_audios.head(1)

,session_id,audio_file,array
0,voip-00d76b791d-20130327_010416,pt344x_0000993_0001219.wav,"[0.000335693359375, 0.000335693359375, 0.00012..."


In [ ]:
# Inner join so that only those that still have an audio are stored
df = df.merge(df_audios[['session_id', 'audio_file']], on=['session_id', 'audio_file'])
len(df)

22266

## 2) Encode labels

In [ ]:
le = LabelEncoder()
df['label_semantics_encoded'] = le.fit_transform(df['label_semantics'])

df.head()

,session_id,turn_index,audio_file,transcript,chat_history_last_9,chat_history_last_9_tokenized,speaker_text_last_9_tokenized,attention_mask,label_semantics,label,label_semantics_encoded
0,voip-00d76b791d-20130327_010416,2,pt344x_0000993_0001219.wav,<user>expensive restaurant in the south part o...,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|area_inform|pricerange,request|food,45
1,voip-00d76b791d-20130327_010416,4,pt344x_0001649_0001680.wav,<user>any<API_call><DB_result>,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|this,inform|area_inform|food_inform|pricerange_offe...,76
2,voip-00d76b791d-20130327_010416,6,pt344x_0002674_0002736.wav,<user>address<API_call><DB_result>,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",request|addr,inform|addr_offer|name,87
3,voip-00d76b791d-20130327_010416,8,pt344x_0003533_0003611.wav,<user>type of food<API_call><DB_result>,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",request|food,inform|area_inform|pricerange_offer|name,101
4,voip-00d76b791d-20130327_011116,2,pt344x_0001082_0001294.wav,<user>lebanese restaurant in the west part of ...,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|area_inform|food,request|food,42


In [ ]:
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

df.head()

,session_id,turn_index,audio_file,transcript,chat_history_last_9,chat_history_last_9_tokenized,speaker_text_last_9_tokenized,attention_mask,label_semantics,label,label_semantics_encoded,label_encoded
0,voip-00d76b791d-20130327_010416,2,pt344x_0000993_0001219.wav,<user>expensive restaurant in the south part o...,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|area_inform|pricerange,request|food,45,55
1,voip-00d76b791d-20130327_010416,4,pt344x_0001649_0001680.wav,<user>any<API_call><DB_result>,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|this,inform|area_inform|food_inform|pricerange_offe...,76,35
2,voip-00d76b791d-20130327_010416,6,pt344x_0002674_0002736.wav,<user>address<API_call><DB_result>,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",request|addr,inform|addr_offer|name,87,34
3,voip-00d76b791d-20130327_010416,8,pt344x_0003533_0003611.wav,<user>type of food<API_call><DB_result>,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",request|food,inform|area_inform|pricerange_offer|name,101,40
4,voip-00d76b791d-20130327_011116,2,pt344x_0001082_0001294.wav,<user>lebanese restaurant in the west part of ...,"<sys>Hello , welcome to the Cambridge restaura...","[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|area_inform|food,request|food,42,55


# Upload to HuggingFace

In [ ]:
# Keep only the necessary columns
cols_to_keep = ['session_id', 'audio_file',
                'chat_history_last_9_tokenized',
                'speaker_text_last_9_tokenized', 'attention_mask',
                'label_semantics', 'label_semantics_encoded',
                'label', 'label_encoded']
df = df[cols_to_keep]

df.head(1)

,session_id,audio_file,chat_history_last_9_tokenized,speaker_text_last_9_tokenized,attention_mask,label_semantics,label_semantics_encoded,label,label_encoded
0,voip-00d76b791d-20130327_010416,pt344x_0000993_0001219.wav,"[50258, 15496, 837, 7062, 284, 262, 14457, 707...","[50258, 50258, 50258, 50258, 50258, 50258, 502...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",inform|area_inform|pricerange,45,request|food,55


In [ ]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['session_id', 'audio_file', 'chat_history_last_9_tokenized', 'speaker_text_last_9_tokenized', 'attention_mask', 'label_semantics', 'label_semantics_encoded', 'label', 'label_encoded', '__index_level_0__'],
    num_rows: 22266
})

In [ ]:
dataset = dataset.remove_columns('__index_level_0__')
dataset

Dataset({
    features: ['session_id', 'audio_file', 'chat_history_last_9_tokenized', 'speaker_text_last_9_tokenized', 'attention_mask', 'label_semantics', 'label_semantics_encoded', 'label', 'label_encoded'],
    num_rows: 22266
})

In [ ]:
dataset.push_to_hub('danielroncel/dstc2_dialogues_input_gpt2')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/23 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/659 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danielroncel/dstc2_dialogues_input_gpt2/commit/2d34badc1b0b1f21357914b1e4d626f793fac25a', commit_message='Upload dataset', commit_description='', oid='2d34badc1b0b1f21357914b1e4d626f793fac25a', pr_url=None, pr_revision=None, pr_num=None)